In [ ]:
# 1. Self-attention by hand
# 2. Self-attention block in pytorch
# 3. GPT, piece-by-piece
# 4. GPU goes rrrr!

# Original code from https://github.com/karpathy/minGPT/tree/master/mingpt

### Step 1: Self-attention by hand

In [29]:
import torch
import math
import torch.nn.functional as F

In [ ]:
#  -- Write the scaled dot product self attention
  # 1. Compute queries, keys, and values
  # 2. Compute dot products
  # 3. Scale the dot products
  # 4. Apply softmax to calculate attentions
  # 5. Weight values by attentions
  # 6. Compute attention weighted features

In [30]:
# Choose random values for the parameters -- sames values as on slide 12, but in pytorch format
# T = 4, C = 6, H = 3
X = torch.tensor([[2,0,0,0,2,1],[0,1,2,0,0,0],[0,0,1,1,0,1],[2,0,0,1,0,1]], dtype=float) # T x C
W_QT = torch.tensor([[1,0,0], [1,1,0], [0,0,1], [0,1,0], [0,0,1], [0,0,1]], dtype=float) # C x H
W_KT = torch.tensor([[0,0,1], [0,1,0], [1,0,0], [0,0,0], [0,0,0], [0,0,-1]], dtype=float) # C x H
W_VT = torch.tensor([[10,0,0], [0,0,10], [0,0,0], [0,10,0], [0,0,0], [0,0,0]], dtype=float) # C x H

In [ ]:
# What does the second dimension of matrices Q and K correspond to?

In [31]:
# compute the weighted attention matrix S
Q = torch.matmul(X, W_QT)
K = torch.matmul(X, W_KT)
QKT = torch.matmul(Q, K.T)
d_k = Q.shape[1]
QKT /= math.sqrt(d_k)
S = F.softmax(QKT, dim=1)


In [32]:
# compute the self-attention matrix A
V = torch.matmul(X, W_VT)
A = torch.matmul(S,V)

In [33]:
# Sanity check. This should return True.
torch.allclose(A.float(), torch.tensor([[10.30759701,  2.83283874,  4.59026201],
        [10.10551833,  2.97334971,  4.50027071],
        [15.03361159,  4.13169018,  2.10990693],
        [ 3.06082018,  1.53041009,  7.70438486]]))

True

### Step 2: Self-attention block in pytorch

In [34]:
import torch
import torch.nn as nn
from torch.functional import F

In [35]:
# do not modify this code

batch_size = 3 # B
block_size = 2 # T
n_embd = 3     # C

In [36]:
torch.set_printoptions(precision=8)

In [40]:
# Build a scaled self-attention head without masked attention and without dropout (i.e. just key, query and values)
# A matrix multiplication is implemented using the nn.Linear() operator with no bias.
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

    def forward (self, x):
        B, T, C = x.shape
        K = self.key(x)
        Q = self.query(x)
        V = self.value(x)
        wei = (Q @ K.transpose(-2, -1)) * (K.shape[-1] ** -0.5)
        wei = F.softmax(wei, dim=-1)
        out = wei @ V
        return out

In [41]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
h = Head(2)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = h(x)
out

tensor([[[-0.46728206,  0.03477209],
         [-0.47425330,  0.05069543]],

        [[-0.38198256,  0.02403206],
         [-0.39846635,  0.02506738]],

        [[-0.29631630,  0.12201241],
         [-0.30199534,  0.12650710]]], grad_fn=<UnsafeViewBackward0>)

In [42]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.46728206,  0.03477207],
         [-0.47425330,  0.05069541]],
        [[-0.38198256,  0.02403205],
         [-0.39846635,  0.02506737]],
        [[-0.29631630,  0.12201238],
         [-0.30199534,  0.12650707]]]))

True

In [46]:
# Add weighted masked attention and dropout. Dropout comes after the softmax and before the multiplication with the value matrix.
# Copy the Head class from the previous exercise and expand upon it.

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.dropout = nn.Dropout(0.2)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # store a persistent buffer for the forward pass

    def forward (self, x):
        B, T, C = x.shape
        K = self.key(x)
        Q = self.query(x)
        V = self.value(x)

        wei = (Q @ K.transpose(-2, -1)) * (K.shape[-1] ** -0.5)

        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        out = wei @ V
        return out

In [47]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
h = Head(2)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = h(x)
out

tensor([[[-0.47424769, -0.20746115],
         [-0.59281659,  0.06336929]],

        [[-0.17730206,  0.01118640],
         [-0.49808297,  0.03133423]],

        [[-0.21627384,  0.03053588],
         [-0.37749419,  0.15813388]]], grad_fn=<UnsafeViewBackward0>)

In [45]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.37939820, -0.16596894],
         [-0.47425330,  0.05069541]],
        [[-0.14184165,  0.00894911],
         [-0.39846635,  0.02506737]],
        [[-0.17301908,  0.02442869],
         [-0.30199534,  0.12650707]]]))


False

In [48]:
# A multi-head attention module contains a list of heads and a linear projection layer.
# The heads are applied to the input and then concatenated along the last dimension, then
# the linear layer is applied. Look at the unit test below to determine the dimensions of
# the linear layer.

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

In [49]:
# do not modify
num_heads = 3
head_size = 2
n_embd = 6

In [50]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
sa = MultiHeadAttention(num_heads=3, head_size=head_size)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = sa(x)

In [51]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.03730504, -0.07006130, -0.27096999,  0.13144857, -0.45049590,
          -0.33217290],
         [-0.06818272, -0.04490501, -0.34806073,  0.15622401, -0.45459983,
          -0.33084857]],
        [[-0.08914752, -0.03846309, -0.36569631,  0.09802882, -0.39963537,
          -0.29225215],
         [-0.04541985,  0.01269679, -0.25225419,  0.08241771, -0.41533324,
          -0.30674040]],
        [[ 0.15234883, -0.08591781, -0.10099770,  0.19886394, -0.49236685,
          -0.43605998],
         [ 0.15418015, -0.01837257, -0.00573672,  0.14228639, -0.48172480,
          -0.40757987]]]))


False

In [52]:
# Add a classical feedforward module: linear -> ReLU -> linear
# The hidden dimension is four times bigger than the input dimension (see Section 3.3 of Attention is All You Need)

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        hidden_dim = 4 * n_embd

        self.linear1 = nn.Linear(n_embd, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.linear2 = nn.Linear(hidden_dim, n_embd)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.linear2(out)
        return out


In [53]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
ff = FeedForward(n_embd)
torch.manual_seed(123) # do not remove this line
x = torch.rand((3,n_embd))
out = ff(x)
out

tensor([[-0.56171656,  0.00671528, -0.08733369,  0.25574088, -0.21769133,
         -0.10549456],
        [-0.55107427,  0.11361884, -0.03093255,  0.03408349, -0.33449104,
          0.08606286],
        [-0.49061197,  0.12174689, -0.05031921, -0.09972109, -0.29760227,
         -0.03681775]], grad_fn=<AddmmBackward0>)

In [54]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[-0.58034140,  0.04641046, -0.10707694,  0.21581653, -0.30361831,
         -0.07352637],
        [-0.48917407,  0.07879593, -0.15972012,  0.17862344, -0.37070659,
         -0.07852858],
        [-0.48530388,  0.09604470, -0.06524836,  0.16611034, -0.35499069,
         -0.08964306]]))

False

In [55]:
# Build a self-attention block
#
#   in -----> LayerNorm -------> multi-head attention -- + ----> LayerNorm -----> FeedForward --- + -----> out
#         |                                              |   |                                    |
#          ----------------------------------------------     ------------------------------------
#
# This architecture is slightly different from Attention is All You Need (or the UDL textbook):
# the layer norm comes before (not after) the attention or feed-forward
#
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.ln1 = nn.LayerNorm(n_embd)
        self.attn = MultiHeadAttention(n_head, head_size)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ffwd = FeedForward(n_embd)

    def forward(self, x):
        attn_out = self.attn(self.ln1(x))
        out = attn_out + x
        ffwd_out = self.ffwd(self.ln2(out))
        out = ffwd_out + out
        return out

In [56]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
bk = Block(n_embd, num_heads)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size,block_size,n_embd))
out = bk(x)
out

tensor([[[-0.28274745, -0.15197974, -0.25662702,  0.96722639, -0.61615402,
           0.46517178],
         [-0.28723764, -0.49662346, -0.33363408,  1.27716315, -0.37670344,
           0.45305294]],

        [[-0.61567461, -0.26879269, -0.29994249,  0.59354782, -0.49588484,
           0.40434441],
         [-0.09587546,  0.02934551,  0.61873144,  0.71886432, -0.02091005,
           0.68256164]],

        [[ 0.41833729,  0.46945339,  0.02747524,  1.36433983,  0.26742882,
          -0.05659816],
         [ 1.32717097,  0.78645396,  1.17269289,  0.45673659,  0.74732399,
           0.44081396]]], grad_fn=<AddBackward0>)

In [57]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.05278997, -0.10863629, -0.09458938,  0.97590691, -0.55101192,
           0.57085067],
         [-0.16924502, -0.45394337, -0.25217158,  1.10904062, -0.34593600,
           0.41432184]],
        [[-0.41515028, -0.30126408, -0.11399293,  0.64651299, -0.51579159,
           0.57017863],
         [-0.02535054,  0.08704096,  0.66524690,  0.69768047,  0.05969021,
           0.69993609]],
        [[ 0.52881187,  0.34458166,  0.31130391,  1.11564195,  0.37998506,
          -0.02971917],
         [ 1.38496208,  0.60325992,  0.99346304,  0.38082033,  0.62151432,
           0.47973478]]]))

False

In [ ]:
## Step 3: Build a mini GPT
#
# - Start from the gpt-problem.py file
# - Add your Head, MultiHeadAttention, FeedForward and Block classes
# - Fill in the GPT class (__init__ and forward methods)
# - Train the network on CPU
# - Train the network on GPU

# For __init__, the GPT model parameters are:
#   - a token embedding table
#   - a positional embedding table
#   - a sequence of Blocks
#   - a layer norm
#   - a linear layer
#
# For forward(), the model consists in:
#   - applying the token embedding table and positional embedding table to the input tensor
#   - adding the two together
#   - applying the blocks, layer norm and linear layer (in that order)
#
# The code comes from hyperparameters that should work well on GPU.  On CPU, you
# will need to reduce the model size significantly.
#
# In pytorch, an learnable embedding table is implemented with nn.Embedding(...)
#
# The token embedding table learns an embedding for each item of the vocabulary. The
# positional embedding table does not depend on the input and learns an embedding
# for each position in the context.